In [4]:
import numpy as np
# from sklearn import linear_model
# from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot as plt
from gurobipy import *
import statsmodels.api as sm

In [ ]:
def inner_opt(Z, Gamma):
    m = Model()
    m.setParam('LogToConsole', 0)
    
    Beta_0 = m.addVars((j for j in range(J)), vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY)
    Beta_1 = m.addVars(((j, i) for j in range(J) for i in range(d)), vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY)
    delta_1 = m.addVars(((j, i) for j in range(J) for i in range(d)), vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY)
    q_1 = m.addVars(((j, i) for j in range(J) for i in range(d)), vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY)
    
    m.setObjective(sum([Z[j, k] * (Beta_0[j] + sum(Beta_1[j, i] * XC[k, i] for i in range(d))) for j in range(J) for k in range(K)]), GRB.MINIMIZE)
    
    for j in range(J):
        m.addConstr(Beta_0[j] + sum([Beta_1[j, i] * mu[j][i] for i in range(d)]) == marginal_estimation[j])
        
    for j in range(J):
        for i in range(d):
            if sign_beta_1[j, i] > 0:
                m.addConstr(Beta_1[j, i] >= 0)
#                 m.addConstr(Beta_1[j, i] <= hat_beta_1[j][i])
            elif sign_beta_1[j, i] < 0:
                m.addConstr(Beta_1[j, i] <= 0)
#                 m.addConstr(Beta_1[j, i] >= hat_beta_1[j][i])
            else:
                m.addConstr(Beta_1[j, i] == 0)
    
    
            m.addConstr(delta_1[j, i] == Beta_1[j, i] - hat_beta_1[j][i])
            m.addConstr(q_1[j, i] == abs_(delta_1[j, i]))
    
    m.addConstr(sum([q_1[j, i] ** 2 for j in range(J) for i in range(d)]) <= Gamma)
        
    m.optimize()
    sol = m.getAttr('X', Beta_1)
    print(sol)

In [ ]:
# robust model

def robust_opt(Gamma):
    m = Model()
    m.setParam('LogToConsole', 0)

    Z = m.addVars(((j, k) for k in range(K) for j in range(J)), vtype=GRB.BINARY, lb=0, ub=1)
    Theta = m.addVars((j for j in range(J)), vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY)
    alpha = m.addVar(vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=0)
    Lambda_1 = m.addVars(((j, i) for j in range(J) for i in range(d)), vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY)
    
    part_0 = alpha * Gamma
    part_1 = sum([Lambda_1[j, i] * hat_beta_1[j][i] for j in range(J) for i in range(d)])

    m.setObjective(sum([Theta[j] * marginal_estimation[j] for j in range(J)]) + part_0 + part_1, GRB.MAXIMIZE)

    for s in range(S):
        ## part 0
        for j in range(J):           
            m.addConstr(Theta[j] == sum([Z[j, k] for k in range(K)]))

        ## part 1
        for j in range(J):
            for i in range(d):
                if sign_beta_1[j, i] == 'NA':
                    m.addConstr(Theta[j] * mu[j][i] + Lambda_1[j, i]== sum([Z[j, k] * XC[k, i] for k in range(K)]))
                else:
                    if sign_beta_1[j, i] > 0:
                        m.addConstr(Theta[j] * mu[j][i] + Lambda_1[j, i] <= sum([Z[j, k] * XC[k, i] for k in range(K)]))
                    if sign_beta_1[j, i] < 0:
                        m.addConstr(Theta[j] * mu[j][i] + Lambda_1[j, i] >= sum([Z[j, k] * XC[k, i] for k in range(K)]) )
           
        m.addConstr(sum([Lambda_1[j, i] ** 2 for j in range(J) for i in range(d)]) <= alpha ** 2)
        
        
    for k in range(K):
        m.addConstr(Z.sum('*', k) == 1)
        
#     m.addConstr(sum([Z[j, k] * 1 for j in range(J) for k in range(K)]) <= K)

    m.optimize()
    
    policy = ['NA'] * K
    sol = m.getAttr('X', Z)
    for j, k in sol:
        if sol[j, k] > 0.9:
             policy[k] = j
    
    return policy, sol